## Script to generate Excel File from FHIR Questionnaires

- python version 3.6+
- convert json to excel first using Flatten, Pandas and Openpyxl libraries and using dict pop to isolate key repeating pieces items to save in separate sheets.

- read meta from meta sheet
- read items from item sheet
- valueset-expansions where applicable


In [334]:
from json import load, dumps
from flatten_json import flatten
import pandas as pd
from csv import writer
import openpyxl
from IPython import display as D



in_path = '/Users/ehaas/Documents/FHIR/Argo-Questionnaire/source/examples/'
in_file = ['ASQ3','AUDIT','DAST','Housing','PHQ9','Sampler']
out_path = ''
q_parts = ['contained','item','text']

### Open Questionnaires as DICT files

In [287]:
def open_file(name): # get files
    with open(f'{in_path}{name}.json') as f:
        q = load(f)
        return(q)

-  Split DICT into Q meta, an unnested list of Q Items, Contained VS meta and Concepts.
-  flatten DICT
-  Display as Pandas
-  Write to Excel files

In [352]:
 for name in in_file:
    q = open_file(name)
    # first separate the the meta part from text contained and items
    contained_q = q.pop('contained',None)
    text_q = q.pop('text',None)
    item_q = q.pop('item', None)
    meta_q = q   
    # use  flatten create a pair of excel sheets
    flat_meta = flatten(meta_q)

    ## now use panda

    df0 = pd.Series(flat_meta).to_frame('value')
    D.display(df0)
 

    ## now use pandas to creates the q.items as an array instead of recursion go three levels deep for now:
    itemlist=[]
    for i in item_q:
        for j in i.pop('item'):
            try:
                for k in j.pop('item'):
                    itemlist.append(k)
            except KeyError:
                pass
            itemlist.append(j)
        itemlist.append(i)


    flat_items = (flatten(d) for d in itemlist)

    df1 = pd.DataFrame(flat_items)
    
    ## now write to excel sheets

    with pd.ExcelWriter(f'{out_path}{name}.xlsx') as writer:
        df0.to_excel(writer, sheet_name='Meta')
        df1.to_excel(writer, sheet_name='Items')
    
   
    D.display(df1.sort_values('linkId'))  #display

    ## now same for contained
    if contained_q:
        for i,contained in enumerate(contained_q):
            contained_exp = contained['expansion'].pop('contains')
            # print(contained_exp)
            flat_vs = flatten(contained)
            df2 = pd.Series(flat_vs).to_frame('value')
            D.display(df2)
            flat_exp = (flatten(d) for d in contained_exp)
            df3 = pd.DataFrame(flat_exp)
            D.display(df3)
            # open previous workbook and add new sheets......
            book = openpyxl.load_workbook(f'{out_path}{name}.xlsx')
            with pd.ExcelWriter(f'{out_path}{name}.xlsx') as writer:
                writer.book = book
                df2.to_excel(writer, sheet_name=f'Cont_VS_{i}_Meta')
                df3.to_excel(writer, sheet_name=f'Cont_VS_{i}_Concepts')
    

,value
code_0_code,asq3
code_0_display,Ages and Stages Questionnaire (ASQ-3) 16 Month...
code_0_system,http://acme.org/codes
contact_0_telecom_0_system,url
contact_0_telecom_0_value,http://argonautwiki.hl7.org
copyright,"Ages &amp; Stages Questionnaires®, Third Editi..."
date,2018-07-26
id,questionnaire-example-asq3
identifier_0_system,http://acme.org/q-identifiers
identifier_0_value,business-identifier5


,extension_0_url,extension_0_valueBoolean,linkId,option_0_extension_0_url,option_0_extension_0_valueDecimal,option_0_valueCoding_code,option_0_valueCoding_display,option_0_valueCoding_system,option_1_extension_0_url,option_1_extension_0_valueDecimal,...,option_1_valueCoding_system,option_2_extension_0_url,option_2_extension_0_valueDecimal,option_2_valueCoding_code,option_2_valueCoding_display,option_2_valueCoding_system,prefix,required,text,type
1,http://hl7.org/fhir/StructureDefinition/questi...,True,G0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,group
0,NaN,NaN,G0.d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,On the following pages are questions about act...,display
5,NaN,NaN,G1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,GROSS MOTOR,group
2,NaN,NaN,G1.4,http://fhir.org/guides/argonaut-questionnaire/...,10.0,yes,YES,http://fhir.org/guides/argonaut-questionnaire/...,http://fhir.org/guides/argonaut-questionnaire/...,5.0,...,http://fhir.org/guides/argonaut-questionnaire/...,http://fhir.org/guides/argonaut-questionnaire/...,0.0,not-yet,NOT YET,http://fhir.org/guides/argonaut-questionnaire/...,4,True,"Does your child move around by walking, rather...",choice
3,NaN,NaN,G1.5,http://fhir.org/guides/argonaut-questionnaire/...,10.0,yes,YES,http://fhir.org/guides/argonaut-questionnaire/...,http://fhir.org/guides/argonaut-questionnaire/...,5.0,...,http://fhir.org/guides/argonaut-questionnaire/...,http://fhir.org/guides/argonaut-questionnaire/...,0.0,not-yet,NOT YET,http://fhir.org/guides/argonaut-questionnaire/...,5,True,Does your child walk well and seldom fall?,choice
4,NaN,NaN,G1.6,http://fhir.org/guides/argonaut-questionnaire/...,10.0,yes,YES,http://fhir.org/guides/argonaut-questionnaire/...,http://fhir.org/guides/argonaut-questionnaire/...,5.0,...,http://fhir.org/guides/argonaut-questionnaire/...,http://fhir.org/guides/argonaut-questionnaire/...,0.0,not-yet,NOT YET,http://fhir.org/guides/argonaut-questionnaire/...,6,True,Does your child climb on an object such as a c...,choice
9,NaN,NaN,G2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,FINE MOTOR,group
6,NaN,NaN,G2.4,http://fhir.org/guides/argonaut-questionnaire/...,10.0,yes,YES,http://fhir.org/guides/argonaut-questionnaire/...,http://fhir.org/guides/argonaut-questionnaire/...,5.0,...,http://fhir.org/guides/argonaut-questionnaire/...,http://fhir.org/guides/argonaut-questionnaire/...,0.0,not-yet,NOT YET,http://fhir.org/guides/argonaut-questionnaire/...,4,True,Does your child help turn pages of a book? (Yo...,choice
7,NaN,NaN,G2.5,http://fhir.org/guides/argonaut-questionnaire/...,10.0,yes,YES,http://fhir.org/guides/argonaut-questionnaire/...,http://fhir.org/guides/argonaut-questionnaire/...,5.0,...,http://fhir.org/guides/argonaut-questionnaire/...,http://fhir.org/guides/argonaut-questionnaire/...,0.0,not-yet,NOT YET,http://fhir.org/guides/argonaut-questionnaire/...,5,True,Does your child throw a small ball with a forw...,choice
8,NaN,NaN,G2.6,http://fhir.org/guides/argonaut-questionnaire/...,10.0,yes,YES,http://fhir.org/guides/argonaut-questionnaire/...,http://fhir.org/guides/argonaut-questionnaire/...,5.0,...,http://fhir.org/guides/argonaut-questionnaire/...,http://fhir.org/guides/argonaut-questionnaire/...,0.0,not-yet,NOT YET,http://fhir.org/guides/argonaut-questionnaire/...,6,True,Does your child stack a small block or toy on ...,choice


,value
code_0_code,72110-0
code_0_display,The Alcohol Use Disorder Identification Test (...
code_0_system,http://loinc.org
contact_0_telecom_0_system,url
contact_0_telecom_0_value,http://argonautwiki.hl7.org
copyright,Copyright 2001 World Health Organization. Term...
date,2018-07-26
id,questionnaire-example-audit
identifier_0_system,http://acme.org/q-identifiers
identifier_0_value,audit


,code_0_code,code_0_system,extension_0_url,extension_0_valueBoolean,linkId,options_display,options_reference,prefix,repeats,required,text,type
12,72110-0,http://loinc.org,http://hl7.org/fhir/StructureDefinition/questi...,True,g1,NaN,NaN,NaN,NaN,True,The Alcohol Use Disorders Identification Test:...,group
0,NaN,NaN,NaN,NaN,g1.d,NaN,NaN,NaN,NaN,True,"Over the last 2 weeks, how often have you been...",display
11,NaN,NaN,http://hl7.org/fhir/StructureDefinition/questi...,True,g1.d2,NaN,NaN,NaN,NaN,True,Scoring the AUDIT Scores for each question ran...,display
1,68518-0,http://loinc.org,NaN,NaN,g1.q1,SAMHSA-6 drinks / Never/Less than monthly/Mont...,#SAMHSA-6,1.,False,True,How often do you have a drink containing alcohol?,choice
10,NaN,NaN,NaN,NaN,g1.q10,"No|Yes, but not in the last year|Yes, during t...",#example-answerlist1,10.,False,NaN,"Has a relative or friend, doctor or other heal...",choice
2,68519-8,http://loinc.org,NaN,NaN,g1.q2,SAMHSA-drinks a day / 1 or 2/3 or 4/5 or 6/7 t...,#SAMHSA-dad,2.,False,True,How many standard drinks containing alcohol do...,choice
3,68520-6,http://loinc.org,NaN,NaN,g1.q3,SAMHSA-6 drinks / Never/Less than monthly/Mont...,#SAMHSA-6,3.,False,True,How often do you have six or more drinks on on...,choice
4,NaN,NaN,NaN,NaN,g1.q4,SAMHSA-6 drinks / Never/Less than monthly/Mont...,#SAMHSA-6,4.,False,True,"During the past year, how often have you found...",choice
5,NaN,NaN,NaN,NaN,g1.q5,SAMHSA-6 drinks / Never/Less than monthly/Mont...,#SAMHSA-6,5.,False,True,"During the past year, how often have you faile...",choice
6,NaN,NaN,NaN,NaN,g1.q6,SAMHSA-6 drinks / Never/Less than monthly/Mont...,#SAMHSA-6,6.,False,True,"During the past year, how often have you neede...",choice


,value
copyright,This content LOINC is copyright 1995 Regenstri...
date,2018-07-04
expansion_identifier,samhsa6-expansion
expansion_timestamp,2018-07-04T14:43:56
expansion_total,5
id,SAMHSA-6
identifier_0_value,samhsa6
meta_profile_0,http://fhir.org/guides/argonaut-questionnaire/...
resourceType,ValueSet
status,active


...

,code,display,extension_0_url,extension_0_valueDecimal,system,version
0,LA6568-5,Not at all,http://hl7.org/fhir/StructureDefinition/values...,0,http://loinc.org,2.64
1,LA6569-3,Several days,http://hl7.org/fhir/StructureDefinition/values...,1,http://loinc.org,2.64
2,LA6570-1,More than half the days,http://hl7.org/fhir/StructureDefinition/values...,2,http://loinc.org,2.64
3,LA6571-9,Nearly every day,http://hl7.org/fhir/StructureDefinition/values...,3,http://loinc.org,2.64


,value
copyright,This content LOINC is copyright 1995 Regenstri...
date,2018-06-27
expansion_identifier,phq10-expansion
expansion_timestamp,2018-06-27T22:27:03
expansion_total,4
id,PHQ-10
identifier_0_value,phq10
meta_profile_0,http://fhir.org/guides/argonaut-questionnaire/...
resourceType,ValueSet
status,active


,code,display,system,version
0,LA6572-7,Not difficult at all,http://loinc.org,2.64
1,LA6573-5,Somewhat difficult,http://loinc.org,2.64
2,LA6575-0,Very difficult,http://loinc.org,2.64
3,LA6574-3,Extremely difficult,http://loinc.org,2.64


,value
code_0_code,foo
code_0_display,Argonaut Questionnaire Sampler
code_0_system,http://acme.org
contact_0_telecom_0_system,url
contact_0_telecom_0_value,http://argonautwiki.hl7.org
copyright,"You can copy, modify, distribute and perform t..."
date,2018-07-26
id,questionnaire-example-sampler
identifier_0_system,http://acme.org/q-identifiers
identifier_0_value,questionnaire-example-sampler


,extension_0_url,extension_0_valueDecimal,linkId,option_0_extension_0_url,option_0_extension_0_valueDecimal,option_0_valueString,option_1_extension_0_url,option_1_extension_0_valueDecimal,option_1_valueString,option_2_extension_0_url,option_2_extension_0_valueDecimal,option_2_valueString,prefix,repeats,required,text,type
1,NaN,NaN,g1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,Header Text,group
0,NaN,NaN,g1.d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,The Argonaut Questionnaire Sampler is designed...,display
19,NaN,NaN,g10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.,NaN,True,url,group
18,http://fhir.org/guides/argonaut-questionnaire/...,11.0,g10.q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True,Provide a link to a video clip where the avera...,url
21,NaN,NaN,g11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.,NaN,True,choice,group
20,NaN,NaN,g11.q1,http://fhir.org/guides/argonaut-questionnaire/...,0.0,10,http://fhir.org/guides/argonaut-questionnaire/...,43.0,11,http://fhir.org/guides/argonaut-questionnaire/...,0.0,12,NaN,False,True,What is the average air speed velocity of a la...,choice
23,NaN,NaN,g12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.,NaN,True,open-choice,group
22,NaN,NaN,g12.q1,http://fhir.org/guides/argonaut-questionnaire/...,0.0,10,http://fhir.org/guides/argonaut-questionnaire/...,43.0,11,http://fhir.org/guides/argonaut-questionnaire/...,0.0,12,NaN,False,True,What is the average air speed velocity of a la...,open-choice
25,NaN,NaN,g13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.,NaN,True,quantity,group
24,http://fhir.org/guides/argonaut-questionnaire/...,3.0,g13.q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True,What is the average air speed velocity of a la...,quantity
